In [66]:
import ast
import astor
class InputVisitor(ast.NodeVisitor):
    def __init__(self):
        self.user_inputs = {}

    def visit_Assign(self, node):
        for child in ast.walk(node.value):
            if isinstance(child, ast.Call):
                if isinstance(child.func, ast.Name):
                    if child.func.id == 'input':
                        for target in node.targets:
                            if isinstance(target, ast.Name):
                                print(target.id)
                                if(str(target.id)  not in self.user_inputs):
                                   self.user_inputs[target.id]=str
                    elif child.func.id == 'int':
                        for target in node.targets:
                            if isinstance(target, ast.Name):
                                self.user_inputs[target.id] = int
                                break
                    elif child.func.id == 'float':
                        for target in node.targets:
                            if isinstance(target, ast.Name):
                                self.user_inputs[target.id] = float
                                break
        self.generic_visit(node)


def type_user_inputs(code_file):
    with open(code_file, 'r') as f:
        code = f.read()
    tree = ast.parse(code)
    visitor = InputVisitor()
    visitor.visit(tree)
    return visitor.user_inputs


In [67]:
type_user_inputs("code_2.py")

knight_hor
knight_column
bishop_hor
bishop_column


{'knight_hor': str,
 'knight_column': int,
 'bishop_hor': str,
 'bishop_column': int}

In [63]:
import ast
import astor

class InputVisitor(ast.NodeTransformer):
    def __init__(self):
        self.user_inputs = {}
        self.param_names = ["param" + str(i) for i in range(1, 11)] # Değişken isimleri

    def visit_Call(self, node):
        if isinstance(node.func, ast.Name) and node.func.id == "input":
            # input fonksiyonunu değiştir
            new_name = self.param_names.pop(0)
            self.user_inputs[new_name] = str
            new_node=ast.Name(id=new_name, ctx=ast.Load())
            return ast.copy_location(new_node, node)
        else:
            return self.generic_visit(node)


def detect_user_inputs(code_file):
    with open(code_file, 'r') as f:
        code = f.read()
    tree = ast.parse(code)
    visitor = InputVisitor()
    visitor.visit(tree)
    new_tree=ast.fix_missing_locations(tree)
    # func_def_node = ast.parse("def my_function(" + ", ".join(visitor.user_inputs.keys()) + "):\n    " + astor.to_source(new_tree).strip())
    # ast.copy_location(func_def_node,new_tree)
    args_arr = [ast.arg(arg=name, annotation=None) for name in visitor.user_inputs.keys()]
    function_node = ast.FunctionDef(
    name="my_function",
    args=ast.arguments(
        args=[ast.arg(arg=name, annotation="str") for name in visitor.user_inputs.keys()],
        vararg=None,
        kwonlyargs=[],
        kw_defaults=[],
        kwarg=None,
        defaults=[]
    ),
    body=new_tree.body,
    decorator_list=[],
    returns=None
)
    with open("final_file1.py", "w") as f:
        f.write(astor.to_source(function_node))
    return visitor.user_inputs


In [64]:
detect_user_inputs("code_2.py")

{'param1': str, 'param2': str, 'param3': str, 'param4': str}